In [1]:
import matplotlib
#matplotlib.use('Agg') # If we don't want saved images printed on output

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import time

from Models.loss import smoothL1

from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input, InputLayer
from tensorflow.keras.layers import Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

#from tensorflow.compat.v1 import ConfigProto
#from tensorflow.compat.v1 import InteractiveSession
#config = ConfigProto()
#config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

import math
import io
import os
from collections import namedtuple
import sys
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd

import copy
import glob

#from vis.visualization import visualize_cam
#import cv2
#FIXIT Needs old Scipy version (1.1.0), need to check integration with Conda and Pip

#import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 

In [2]:
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import numpy as np
from io import StringIO
from io import BytesIO

In [3]:
from tf_keras_vis.utils import num_of_gpus

from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize

from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

_, gpus = num_of_gpus()
print('{} GPUs'.format(gpus))

2 GPUs


In [4]:
"""
Augmentation System
By: Tyler Staudinger
Using CPU based augmentation from a dataset loaded into memory incurs about 10% additional per epoch 
processing time
Copyright 2018 The Boeing Company
"""

from imgaug import augmenters as iaa
import numpy as np
import cv2

#Apply various augmentations from imgaug
def blur():
    blurer=[iaa.GaussianBlur((0, 3.0)),iaa.AverageBlur(k=(2, 7)),iaa.MedianBlur(k=(3, 11))]
    return blurer[np.random.randint(0,3)]

#Dropout may not be appropriate, so its not currently used
def dropout():
    dropper=[iaa.Dropout((0.01, 0.1), per_channel=0.5), iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2)]
    return dropper[np.random.randint(0,2)]

def noise():
    return iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)

def contrast():
    return iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5)

def greyscale():
    return iaa.Grayscale(alpha=(0.0, 1.0))

def invert():
    return iaa.Invert(0.05, per_channel=True)

def hue():
    return iaa.AddToHueAndSaturation((-20, 20))

def add(): 
    return iaa.Add((-10, 10), per_channel=0.5)

def multiply():
    return iaa.Multiply((0.5, 1.5), per_channel=0.5)

def sharpen(): 
    return iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))

def emboss(): 
    return iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0))
   
def translate():
    return iaa.Affine(translate_percent={"x": (0.0, 0.0), "y": (-0.03, 0.03)},mode='edge')
    
def rotate():
    return iaa.Affine(rotate=(-3, 3),mode='edge')    

   
def augment(imgs,max_augs,affine=False,debug=False):
    """Applys a series of augmentations to an image
    # Arguments
        imgs: the input images
        max_augs: the maximum number of augmentations to apply
    # Returns
        Augmented images
    """
    augmented=[]
    for img in imgs:
        if affine:
            affine_augs=[translate(),rotate()]            
            seq_affine=iaa.Sequential(affine_augs)
            img=seq_affine.augment_image(img)
        #Choose a number of augmentations to use
        num_aug=np.random.randint(0,max_augs+1)
        #set of augmentations to use
        augs=[blur(),noise(),contrast(),greyscale(),invert(),hue(),add(),multiply(),sharpen(),emboss()]
        #apply augmentations
        seq=iaa.Sequential(list(np.random.choice(augs,num_aug)))
        aug_img=seq.augment_image(img)
        
        #For Debugging of augmentations
        if debug:
            cv2.imshow('frame',aug_img)
            cv2.waitKey(1000)
        augmented.append(aug_img)
        
    return np.asarray(augmented)
    
    
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size, noaugs=False, num_aug=5, affine=False, height=None, width=None): 
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.noaugs     = noaugs
        self.num_aug    = num_aug
        self.affine     = affine
        self.height     = height
        self.width      = width
        
    def __getitem__(self, idx):
        
        #indecies=np.random.randint(0,self.X.shape[0],self.batch_size)
            
        if self.noaugs:
            images=self.X[idx * self.batch_size:(idx + 1) *self.batch_size]/255.0
            target=self.y[idx * self.batch_size:(idx + 1) *self.batch_size]
            #Normalize cte with max magnitude of 8 meters
            target[:,0]/=8.0 #16.0
            #Normalize heading with max magnitude of 35 degrees
            target[:,1]/=35.0 #30.0
            return images,target
        imgs=augment(self.X[idx * self.batch_size:(idx + 1) *self.batch_size],self.num_aug,self.affine)/255.0
        
        target=self.y[idx * self.batch_size:(idx + 1) *self.batch_size]
        #Normalize cte with max magnitude of 16 meters
        target[:,0]/=8.0
        #Normalize heading with max magnitude of 35 degrees
        target[:,1]/=35.0
        return imgs,target
        
        
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)

## Loading the Data and Model

In [5]:
LAYER = 4
if LAYER == 1:
    layer = 'dense_1'
elif LAYER == 2:
    layer = 'dense_2'
elif LAYER == 3:
    layer = 'dense_3'
else:
    layer = 'ALL'
    
PLOT_DATA_BOEING = False
PLOT_DATA_KJ = False

BOEING_TRAIN = True
BOEING_TEST = False

MODEL = 'SimpleModel'
#MODEL = 'FullyConnectedModel'

In [6]:
## Loading the data
eval_folder = '../20200630_TaxiNet_data/'
eval_folder_kjtrain = '/home/dgopinat/Work/KJ-TaxinetArtifacts/data_train/'
eval_folder_kjtest = '/home/dgopinat/Work/KJ-TaxinetArtifacts/data_val/'

#Loading the data
if BOEING_TRAIN:
    x=np.load(eval_folder+'X_train.npy')
    y=np.load(eval_folder+'Y_train.npy')
    x_test=np.load(eval_folder+'X_test.npy')
    y_test=np.load(eval_folder+'Y_test.npy')

elif BOEING_TEST:
    x=np.load(eval_folder+'X_small_test.npy')
    y=np.load(eval_folder+'Y_small_test.npy')
    x_train = np.load(eval_folder+'X_train_upd_lay1.npy')
    y_train = np.load(eval_folder+'Y_train_upd_lay1.npy')
    print('X TRAIN SHAPE:', x_train.shape)
    print('Y TRAIN SHAPE:', y_train.shape)

print("Shape of X:", x.shape)
print("Shape of X test:", x_test.shape)


Shape of X: (13885, 200, 360, 3)
Shape of X test: (2777, 200, 360, 3)


In [7]:
#Loading the model
if MODEL == 'SimpleModel':
    model_path = './Training_Runs/TaxiNet-Simple/SimpleModel.h5'
elif MODEL == 'MobileNetModel':
    model_path = './Training_Runs/TaxiNet-MobileNet/MobileNetV2.h5'
else:
    model_path = './Training_Runs/TaxiNet-FullyConnected/FullyConnectedModel.h5'

print('Loading the ORIG model and printing the summary:')
model=load_model(model_path,custom_objects={'smoothL1':smoothL1}) 
model.summary()

Loading the ORIG model and printing the summary:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 178, 24)       1800      
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 178, 24)       96        
_________________________________________________________________
activation_1 (Activation)    (None, 98, 178, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 87, 36)        21600     
_________________________________________________________________
batch_normalization_2 (Batch (None, 47, 87, 36)        144       
_________________________________________________________________
activation_2 (Activation)    (None, 47, 87, 36)        0         
_________________________________________________________________
conv2

In [12]:
delimiter = ','
try:
        data = np.genfromtxt('Boeing_Taxinet_annotations.txt', delimiter=delimiter)#, dtype='|S128')
except:
        import pandas
        data = pandas.read_csv('Boeing_Taxinet_annotations.txt',
                               header=None,
                               delimiter=delimiter,
                               na_filter=True,
                               dtype=str).fillna(fill_na).values
        
print(data.shape)
dataset = list(data)


(299, 5)
18.0
0.0
0.0
0.0
1.0


### Testing Inputs with Boeing-TaxiNet

In [13]:
outputs_test   = []
inference_time = []

for img in x_test:
    t=time.time()
    outputs_test.append(model.predict(np.expand_dims(img/255.0,axis=0)))
    inference_time.append((1/(time.time()-t)))

outputs_test=np.squeeze(np.asarray(outputs_test))
inference_time=np.squeeze(np.asarray(inference_time))

#Scale the predictions
outputs_cte=outputs_test[:,0]*8.0
outputs_heading=outputs_test[:,1]*35.0

error_test=np.squeeze(y_test[:,0])-outputs_cte
    
print(" ORIGINAL MODEL")
print("Cross Track Error")
abs_error_test=np.abs(error_test)
print('MeanAbsoluteError: '+str(np.mean(abs_error_test)))
print('MedianAbsoluteError: '+str(np.median(abs_error_test)))
print('StandardDeviation: '+str(np.std(abs_error_test)))
print('MeanInferenceTime: '+str(np.mean(inference_time)))
print('MaxAbsoluteError: '+str(np.max(abs_error_test)))

error_test=np.squeeze(y_test[:,1])-outputs_heading
    
print("Heading Error")
#Print error metrics
abs_error_test=np.abs(error_test)
print('MeanAbsoluteError: '+str(np.mean(abs_error_test)))
print('MedianAbsoluteError: '+str(np.median(abs_error_test)))
print('StandardDeviation: '+str(np.std(abs_error_test)))
print('MaxAbsoluteError: '+str(np.max(abs_error_test)))
    

 ORIGINAL MODEL
Cross Track Error
MeanAbsoluteError: 0.36629044055199295
MedianAbsoluteError: 0.2837369384839286
StandardDeviation: 0.32818068833292025
MeanInferenceTime: 60.7564324858878
MaxAbsoluteError: 3.8953489884122146
Heading Error
MeanAbsoluteError: 1.6450901653917946
MedianAbsoluteError: 1.4151208355401792
StandardDeviation: 1.270872181246601
MaxAbsoluteError: 9.824763055083636


In [ ]:
# This function returns the actual and ideal outputs 
# When Only Feature = True, all inputs within Threshold 1 and Thereshold 2 are labelled '1', '0' otherwise
# When Only Prop = True, inputs are labelled '1' if they satisfy the correctness constraints, '0' otherwise
# When Both Feature and Prop are True, inputs which are not within intersection and sat correct constraint -> label 0, 
# inputs which are within intersection and sat correct constraint -> label 1, inputs which are not within intersection 
# and dont sat correct constraint -> label 2, and inputs which are within intersection and dont sat correct constraint -> label3

def get_prediction(inputs, tensor = None, LAB=y_test, Feature=False, Prop=False,TEST=False,THRESHOLD1=None,THRESHOLD2=None):
    if ((Feature == False) and (Prop==True)):
        if tensor == "ALL":
            full_list = []
            for tensor in ['dense_1', 'dense_2', 'dense_3']:
                layer_name = tensor
                layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
                layer_outputs_test = []
                for img in inputs:
                    layer_result = (layer_model.predict(np.expand_dims(img/255.0,axis=0))>0).astype('int')
                    #layer_output = layer_model.predict(np.expand_dims(img/255.0,axis=0))[0]
                    layer_outputs_test.append(layer_result[0])

                full_list.append(layer_outputs_test)

            new_list = [list(full_list[0][i])+list(full_list[1][i])+list(full_list[2][i]) for i in range(len(inputs))]

            layer_outputs_test = np.asarray(new_list)
            return layer_outputs_test

        if tensor is None or tensor == 'dense_4':
            outputs_test = []

            for img in inputs:
                outputs_test.append(model.predict(np.expand_dims(img/255.0,axis=0)))

            outputs_test    = np.squeeze(np.asarray(outputs_test))

            #Scale the predictions
            outputs_cte     = outputs_test[:,0]*8.0
            outputs_heading = outputs_test[:,1]*35.0

             #Absolute value of the predictions
            abs_outputs_cte = np.abs(outputs_cte)
            abs_outputs_heading = np.abs(outputs_heading)

            print(np.shape(inputs))
            print(np.shape(LAB))
            print(np.shape(outputs_cte))
           
            #Absolute Error between Ground Truth and Prediction for Cross Track Error
            if (TEST == False):
                error_cte=np.squeeze(y[:,0])-outputs_cte
            else:
                error_cte=np.squeeze(LAB[:,0])-outputs_cte

            abs_error_cte=np.abs(error_cte)

            #Absolute Error between Ground Truth and Prediction for Heading Error
            if (TEST == False):
                error_heading     = np.squeeze(y[:,1])-outputs_heading
            else:
                error_heading     = np.squeeze(LAB[:,1])-outputs_heading

            abs_error_heading = np.abs(error_heading)

            ret = []
            for i in range(0,len(abs_outputs_cte)):
                #if (abs_outputs_cte[i] <= 10.0 and abs_error_cte[i] <=1.0):# TEST
                #if (abs_error_cte[i] < 0.4): # TEST
                if ((abs_error_cte[i] < 0.5) and (abs_error_heading[i] < 2.0)): 
               # if (abs_error_heading[i] <= 5.0): 
                    ret.append(1)
                else:
                    ret.append(0)

            return abs_outputs_cte, ret
            #return (abs_error_heading<=10.0).astype('int')
            #return (abs_error_cte<=1.5).astype('int')
        else:
            layer_name = tensor
            layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
            layer_outputs_test = []
            for img in inputs:
                layer_result = (layer_model.predict(np.expand_dims(img/255.0,axis=0))>0).astype('int')
                #layer_output = layer_model.predict(np.expand_dims(img/255.0,axis=0))[0]
                layer_outputs_test.append(layer_result[0])

            layer_outputs_test = np.asarray(layer_outputs_test)

            #Scale the predictions        
            return layer_outputs_test
        
    if ((Feature == True) and (Prop==False)):
        
        if tensor is None or tensor == 'dense_4':
            
            outputs_test = []

            for img in inputs:
                outputs_test.append(model.predict(np.expand_dims(img/255.0,axis=0)))

            outputs_test    = np.squeeze(np.asarray(outputs_test))

            #Scale the predictions
            outputs_cte     = outputs_test[:,0]*8.0
            outputs_heading = outputs_test[:,1]*35.0

             #Absolute value of the predictions
            abs_outputs_cte = np.abs(outputs_cte)
            abs_outputs_heading = np.abs(outputs_heading)
            
            print(np.shape(inputs))
            print(np.shape(LAB))
            print(np.shape(outputs_cte))
           

            #Absolute Error between Ground Truth and Prediction for Cross Track Error
            if (TEST == False):
                error_cte=np.squeeze(y[:,0])-outputs_cte
            else:
                error_cte=np.squeeze(LAB[:,0])-outputs_cte

            abs_error_cte=np.abs(error_cte)

            #Absolute Error between Ground Truth and Prediction for Heading Error
            if (TEST == False):
                error_heading     = np.squeeze(y[:,1])-outputs_heading
            else:
                error_heading     = np.squeeze(LAB[:,1])-outputs_heading

            abs_error_heading = np.abs(error_heading)

            ret = []
            for indx in range(0,len(abs_outputs_cte)):
                if (indx <= THRESHOLD1) or (indx > THRESHOLD2):
                    ret.append(0)
                else:
                    ret.append(1)

            return abs_outputs_cte, ret
        
        
    if ((Feature == True) and (Prop==True)):

        if tensor is None or tensor == 'dense_4':
         
            outputs_test = []

            for img in inputs:
                outputs_test.append(model.predict(np.expand_dims(img/255.0,axis=0)))

            outputs_test    = np.squeeze(np.asarray(outputs_test))

            #Scale the predictions
            outputs_cte     = outputs_test[:,0]*8.0
            outputs_heading = outputs_test[:,1]*35.0

             #Absolute value of the predictions
            abs_outputs_cte = np.abs(outputs_cte)
            abs_outputs_heading = np.abs(outputs_heading)

            print(np.shape(inputs))
            print(np.shape(LAB))
            print(np.shape(outputs_cte))
            
            #Absolute Error between Ground Truth and Prediction for Cross Track Error
            if (TEST == False):
                error_cte=np.squeeze(y[:,0])-outputs_cte
            else:
                error_cte=np.squeeze(LAB[:,0])-outputs_cte

            abs_error_cte=np.abs(error_cte)
            
            

            #Absolute Error between Ground Truth and Prediction for Heading Error
            if (TEST == False):
                error_heading     = np.squeeze(y[:,1])-outputs_heading
            else:
                error_heading     = np.squeeze(LAB[:,1])-outputs_heading

            abs_error_heading = np.abs(error_heading)

            ret = []
            for indx in range(0,len(abs_outputs_cte)):
                #if (abs_outputs_cte[i] <= 10.0 and abs_error_cte[i] <=1.0):# TEST
                #if (abs_error_cte[i] < 0.4): # TEST
                if ((abs_error_cte[indx] <= 0.5) and (abs_error_heading[indx] <= 2.0)): 
               # if (abs_error_cte[indx] <= 5.0): 
                    if ((indx <= THRESHOLD1) or (indx > THRESHOLD2)):
                            ret.append(0)
                    else:
                            ret.append(1)
                else:
                    if ((indx <= THRESHOLD1) or (indx > THRESHOLD2)):
                            ret.append(2)
                    else:
                            ret.append(3)


            return abs_outputs_cte, ret
         

def fingerprint_suffix(inps,ten):
    return (get_prediction(inps, tensor=ten)>0.0).astype('int')



def get_prediction_vals(inputs, tensor = None, TEST=False):
    
    if tensor == "ALL":
        full_list = []
        for tensor in ['dense_1', 'dense_2', 'dense_3']:
            layer_name = tensor
            layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
            layer_outputs_test = []
            for img in inputs:
                layer_result = (layer_model.predict(np.expand_dims(img/255.0,axis=0)))
                layer_outputs_test.append(layer_result[0])

            full_list.append(layer_outputs_test)
        
        new_list = [list(full_list[0][i])+list(full_list[1][i])+list(full_list[2][i]) for i in range(len(inputs))]
            
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    
    if tensor is None or tensor == 'dense_4':
        outputs_test = []
        
        for img in inputs:
            outputs_test.append(model.predict(np.expand_dims(img/255.0,axis=0)))

        outputs_test    = np.squeeze(np.asarray(outputs_test))

        #Scale the predictions
        outputs_cte     = outputs_test[:,0]*8.0
        outputs_heading = outputs_test[:,1]*35.0
        
         #Absolute value of the predictions
        abs_outputs_cte = np.abs(outputs_cte)
        abs_outputs_heading = np.abs(outputs_heading)
        
        #Absolute Error between Ground Truth and Prediction for Cross Track Error
        if (TEST == False):
            error_cte=np.squeeze(y[:,0])-outputs_cte
        else:
            error_cte=np.squeeze(y_test[:,0])-outputs_cte
            
        abs_error_cte=np.abs(error_cte)

        #Absolute Error between Ground Truth and Prediction for Heading Error
        if (TEST == False):
            error_heading     = np.squeeze(y[:,1])-outputs_heading
        else:
            error_heading     = np.squeeze(y_test[:,1])-outputs_heading
            
        abs_error_heading = np.abs(error_heading)
        
        ret = []
        for i in range(0,len(abs_outputs_cte)):
            #if (abs_outputs_cte[i] <= 10.0 and abs_error_cte[i] <=1.0):# TEST
            #if (abs_error_cte[i] < 0.4): # TEST
           # if ((abs_error_cte[i] <= 1.0)):# and (abs_error_heading[i] <= 5.0)): 
            if (abs_error_heading[i] <= 5.0): 
                ret.append(1)
            else:
                ret.append(0)
                
        return abs_outputs_cte, ret
        #return (abs_error_heading<=10.0).astype('int')
        #return (abs_error_cte<=1.5).astype('int')
    else:
        layer_name = tensor
        layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
        layer_outputs_test = []
        for img in inputs:
            layer_result = (layer_model.predict(np.expand_dims(img/255.0,axis=0)))
            layer_outputs_test.append(layer_result[0])

        layer_outputs_test = np.asarray(layer_outputs_test)

        #Scale the predictions        
        return layer_outputs_test

def fingerprint_suffix_vals(inps,ten):
    return (get_prediction_vals(inps, tensor=ten))

def fingerprint_signature(inps, layer_name = 'dense_1'):
    layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    layer_result = (layer_model.predict(np.expand_dims(img/255.0,axis=0))>0).astype('int')[0]
    return layer_result

In [ ]:
# The `output` variable refer to the output of the model,
# so, in this case, `output` shape is `(1, 2)` i.e., (samples, classes).

#Loss and Model Modifier for middle layers
def loss_gen_sum(node_list):
    def loss(output):
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_avg_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss

def loss_gen_separate(node_list):
    def loss(output):
        loss_val = tuple([output[ind][i] for ind,i in enumerate(node_list)])
        return loss_val
    return loss

def model_modifier_mid_layer(current_model):
    target_layer = current_model.get_layer(name=layer)
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

def loss_crosstrack(output):
    return (output[0][0])

def loss_heading(output):
    return (output[0][1])

def no_loss(output):
    return (output[0][0] * 0)

In [ ]:
FONT_PATH  = '/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed.ttf'
IMAGE_SIZE = 28

def mnist_to_rgb(mnist_img):
  """
  Transformsn an MNIST image (shape: <784>) to a grayscale
  RGB image (shape: <28,28,3>)
  """
  return mnist_img
  #pixel_array = mnist_img.reshape(200, 360, 3)  # shape: 28,28
  #rgb_image = np.transpose([pixel_array,pixel_array,pixel_array], axes=[1,2,0])
  #return rgb_image

def pil_img(a):
  '''Returns a PIL image created from the provided RGB array.
  '''
  a = np.uint8(a)
  return PIL.Image.fromarray(a)

def mnist_to_pil_img(inp):
  rgb_inp = 255*inp#255*mnist_to_rgb(inp)
  vis_inp = pil_img(rgb_inp)
  return vis_inp  

def pil_fig(fig):
  # Returns a PIL image obtained from the provided PLT figure.
  buf = io.BytesIO()
  fig.savefig(buf, format='png')
  plt.close(fig)
  buf.seek(0)
  img = PIL.Image.open(buf)
  return img

def show_img(img, fmt='jpeg'):
  '''Displays the provided PIL image
  '''
  #f = StringIO()
  f = BytesIO()
  img.save(f, fmt)
  display(Image(data=f.getvalue()))
 
def show_mnist_img(mnist_img):
  show_img(pil_img(255*mnist_to_rgb(mnist_img)))
  
def gray_scale(img):
  '''Converts the provided RGB image to gray scale.
  '''
  img = np.average(img, axis=2)
  return np.transpose([img, img, img], axes=[1,2,0])

def normalize(attrs, ptile=99):
  '''Normalize the provided attributions so that they fall between
     -1.0 and 1.0.
  '''
  h = np.percentile(attrs, ptile)
  l = np.percentile(attrs, 100-ptile)
  if (max(abs(h), abs(l))) == 0.0:
    return np.clip(attrs/1.0, -1.0, 1.0)  
  else:
    return np.clip(attrs/max(abs(h), abs(l)), -1.0, 1.0)  


def pil_text(strs, shape, start_h=10, start_w=10, font_size=18, color=(0, 0, 0)):
  # Returns a PIL image with the provided text.
  img = pil_img(255*np.ones(shape))
  draw = PIL.ImageDraw.Draw(img)
  font = PIL.ImageFont.truetype(FONT_PATH, font_size)
  h = start_h
  for s in strs: 
    draw.text((start_w,h), s, fill=color, font=font)
    h = h + 30
  return img

def combine(imgs, horizontal=True):
  # Combines the provided PIL Images horizontally or veritically
  if horizontal:
    w = np.sum([img.size[0]+10 for img in imgs])
    h = np.max([img.size[1] for img in imgs])
  else:
    w = np.max([img.size[0] for img in imgs])
    h = np.sum([img.size[1]+10 for img in imgs])
  final_img = PIL.Image.new('RGB', (w, h), color='white')
  pos = 0
  for img in imgs:
    if horizontal:
      final_img.paste(im=img, box=(pos,0))
      pos = pos+img.size[0]+10
    else:
      final_img.paste(im=img, box=(0,pos))
      pos = pos+img.size[1]+10
  return final_img

def visualize_attrs(img, attrs, ptile=99):
  '''Visualizes the provided attributions by first aggregating them
    along the color channel to obtain per-pixel attributions and then
    scaling the intensities of the pixels in the original image in
    proportion to absolute value of these attributions.

    The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = abs(attrs)
  attrs = np.clip(attrs/np.percentile(attrs, ptile), 0,1)
  vis = img*attrs
  return pil_img(vis)
  
  
R=np.array([255,0,0])
G=np.array([0,255,0])
B=np.array([0,0,255])

def visualize_attrs2(img, attrs, pos_ch=G, neg_ch=R, ptile=99):
  '''Visualizes the provided attributions by first aggregating them
     along the color channel and then overlaying the positive attributions
     along pos_ch, and negative attributions along neg_ch.

     The provided image and attributions must of shape (224, 224, 3).
  '''
 
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
 
  attrs = gray_scale(attrs)
  attrs = normalize(attrs, ptile)   
  
  pos_attrs = attrs * (attrs >= 0.0)
  #pos_attrs = pos_attrs1 * (abs(pos_attrs1) >= threshold)
  neg_attrs = -1.0 * attrs * (attrs < 0.0)
  #neg_attrs = -1.0 * neg_attrs1 * (abs(neg_attrs1) >= threshold)
  attrs_mask = pos_attrs*pos_ch + neg_attrs*neg_ch
  vis = 0.3*gray_scale(img) + 0.7*attrs_mask
  return pil_img(vis)

In [ ]:
def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def get_suffix_cluster_vals(neuron_ids, neuron_sig,suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  cls_list = []
  for inp_indx in range(0,len(suffixes)):
    val = suffixes[inp_indx]
    neu_indx = 0
    fnd = 1
    for indx in range(0,len(neuron_sig)):
        if (indx %2 == 0):
            oper = neuron_sig[indx]
            continue
        thres = neuron_sig[indx]
        neu = neuron_ids[neu_indx]
        neu_indx = neu_indx + 1
        if (((oper =='<=') and (val[neu] > thres)) or ((oper == '>') and (val[neu] <= thres))):
            fnd = 0
            break
            
    if (fnd == 1):
        cls_list.append(inp_indx)
            

  return cls_list
  

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True

def is_misclassified(i):
  #print(train_predictions[i])
  #print(y[i][0])
  return False
  #return (1 if train_predictions[i]>0.0 else 0) != (1 if y[i][0]>0.0 else 0)

def visualize_conductances(img, label, layer, neuron_ids, only_on=False):
  # Visualize the conductances for the provided image.
  # Args:
  # - img: the provided mnist image
  # - label: prediction label w.r.t. conductance must be computed
  # - neuron_ids: list of neurons indices from the suffix tensor for which
  #    conductances must be computed.
  # - only_on: If True then conductance is computed only for those neurons
  #    that are on for the given image. 
  vis = [mnist_to_pil_img(img)]
  suffix = fingerprint_suffix([img], layer)
  sumigc = 0.0
  for i, id in enumerate(neuron_ids):
    if only_on and suffix[i] != 1:
      continue  
    igc = conductance(img, label, neuron_id=id)
    # igc = conductances[id]
    sumigc = sumigc + igc
  
  
  avgigc = sumigc / len(neuron_ids)
  maxval = abs(max(avgigc, key=abs))
  minval = abs(min(avgigc, key=abs))
  threshold = (maxval - minval)/2.0
  print("MAX ATR:", maxval, "MIN ATR:", minval, "THRESH:", threshold)
  avgigc = 1.0 * avgigc * (abs(avgigc) >= threshold)
  
  
  vis.append(visualize_attrs2(255*mnist_to_rgb(img), mnist_to_rgb(avgigc)))
  return combine(vis)

def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)
    
  imgs_suffixes = fingerprint_signature(imgs,'dense_1')
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig, imgs_suffixes)
    
  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
    paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def get_all_invariants_vals(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
    #paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    #paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    paths = [[[feature[node]] + p[0],['<='] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0],['>'] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, suffixes, show_samples=False):
  neuron_sig = suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")
  
  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])
  
  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))
  

def describe_invariants_all_labels1(all_invariants,prevlayer,layer,suffixes, COMMON=False, DEC_PREFX= False):
  print("PRINTING PURE RULES WITH SUPPORT MORE THAN 1000 FOR EVERY LABEL:");
  for cl, invs in all_invariants.items():
    if (cl == -1):
      continue
    
    for indx in range (0, 5):
    #len(invs)):
      inv = invs[indx]
      cls = get_suffix_cluster(inv[0],inv[1],suffixes)
      
      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 1000):
        continue
      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"), Support:",inv[2],", Num misclassified", len([i for i in cls if is_misclassified(i)]));

      ##invoke_marabou_chk(LAYER,neurons,signature,cl)

      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return
  
def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)
    
    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return
    
def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])
            
    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]
    
       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break
    
       if (match == 0):
          prefixes.append(dec_pref)
    
    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return

def describe_all_invariants(all_invariants):
  df = []
  for cl, invs in all_invariants.iteritems(): 
    inv = invs[0]
    clus = get_suffix_cluster(inv[0],inv[1])
    #print(len(clus))
    misCl = 0
    for i in range(0,len(clus)):
      indx = clus[i]
      if (is_misclassified(indx) == True):
        misCl = misCl + 1
    print('class:',cl,',masSup:',inv[2],',#misCl:',misCl)
          

In [ ]:
layer1 = 'dense_1'
layer2 = 'dense_2'
layer3 = 'dense_3'

### TEST SET
# Get the fingerprint (neuron values) after dense_1 and the output labels for correctness property
print("LAYER 1:")
test_suffixes1 = fingerprint_suffix_vals(x_test, layer1)
print(np.shape(x_test),np.shape(y_test))
print("Suffixes computed for all test data")
actual_preds1, test_predictions1 = get_prediction(x_test,Feature=False,Prop=True,TEST=True)
print("Predictions computed for all test data")

print("Class 1:", list(test_predictions1).count(1))
print("Class 0:", list(test_predictions1).count(0))

x_test_full_seq = []
y_test_full_seq = []
x_test_seq = []
y_test_seq = []
x_val_seq = []
y_val_seq = []

cnt = 0
threshold1 = -1
threshold2 = -1
for ix in range(0, len(x_test)):
    num = ix/100.0
    if (num > 19 and num < 24):
        x_test_full_seq.append(x_test[ix])
        y_test_full_seq.append(y_test[ix])
        if (num == 21.54):
            threshold1 = cnt
        if (num == 22.54):
            threshold2 = cnt
        cnt =cnt + 1
        
import random
length = len(x_test_full_seq)
ind1 = 0
valinds = []
while (ind1 < (length)/2):
    ix1 = random.randint(0,length)
    while(ix1 in valinds):
        ix1 = random.randint(0,length)
    valinds.append(ix1)
    ind1 = ind1 + 1

feat_val_nums = []
feat_test_nums = []
for ind1 in range(0,length):
    if (ind1 in valinds):
        x_val_seq.append(x_test_full_seq[ind1])
        y_val_seq.append(y_test_full_seq[ind1])
        if (ind1 >= threshold1 and ind1 <= threshold2):
            feat_val_nums.append(len(x_val_seq)-1)
    else:
        x_test_seq.append(x_test_full_seq[ind1])
        y_test_seq.append(y_test_full_seq[ind1])
        if (ind1 >= threshold1 and ind1 <= threshold2):
            feat_test_nums.append(len(x_test_seq)-1)

x_val_seq = np.array(x_val_seq)
y_val_seq = np.array(y_val_seq)
x_test_seq = np.array(x_test_seq)
y_test_seq = np.array(y_test_seq)

feat_val_nums.sort()
feat_test_nums.sort()
print("VAL NUM:", len(x_val_seq), "TEST NUM:", len(x_test_seq))
print("FEATURE VAL NUM:", feat_val_nums, "FEATURE TEST NUM:", feat_test_nums)

feat_val_nums = np.array(feat_val_nums)
feat_test_nums = np.array(feat_test_nums)

threshold1 = feat_test_nums[0]
feat_len = len(feat_test_nums)-1
threshold2 = feat_test_nums[feat_len]

Vthreshold1 = feat_val_nums[0]
feat_len = len(feat_val_nums)-1
Vthreshold2 = feat_val_nums[feat_len]


# Get the fingerprint (neuron values) after dense_1 and the output labels for Feature Intersection
test_suffixes1F = fingerprint_suffix_vals(x_test_seq, layer1)
print("FEATURE Suffixes computed for all test data")
actual_preds1F, test_predictions1F = get_prediction(x_test_seq,LAB=np.array(y_test_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=threshold1,THRESHOLD2=threshold2)
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(test_predictions1F).count(1))
print("Class 0 F:", list(test_predictions1F).count(0))

# Get the fingerprint (neuron values) after dense_1 and the output labels for Feature Intersection and correctness prop 
test_suffixes1FT = fingerprint_suffix_vals(x_test_seq, layer1)
print("FEATURE AND PROP Suffixes computed for all test data")
actual_preds1FT, test_predictions1FT = get_prediction(x_test_seq,LAB=np.array(y_test_seq),Feature=True,Prop=True,TEST=True,THRESHOLD1=threshold1,THRESHOLD2=threshold2)
print("FEATURE AND PROP Predictions computed for all test data")

print("Class 0 FT:", list(test_predictions1FT).count(0))
print("Class 1 FT:", list(test_predictions1FT).count(1))
print("Class 2 FT:", list(test_predictions1FT).count(2))
print("Class 3 FT:", list(test_predictions1FT).count(3))

# Get the fingerprint (neuron values) after dense_1 and the output labels for correctness property
test_suffixes1CI = fingerprint_suffix_vals(x_test_seq, layer1)
print("PROP Suffixes computed for all test data")
actual_preds1CI, test_predictions1CI = get_prediction(x_test_seq,LAB =np.array(y_test_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(test_predictions1CI).count(1))
print("Class 0:", list(test_predictions1CI).count(0))

### VALIDATION SET

val_suffixes1F = fingerprint_suffix_vals(x_val_seq, layer1)
print("FEATURE Suffixes computed for all val data")
Vactual_preds1F, val_predictions1F = get_prediction(x_val_seq,LAB=np.array(y_val_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=Vthreshold1,THRESHOLD2=Vthreshold2)
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(val_predictions1F).count(1))
print("Class 0 F:", list(val_predictions1F).count(0))


val_suffixes1CI = fingerprint_suffix_vals(x_val_seq, layer1)
print("PROP Suffixes computed for all val data")
Vactual_preds1CI, val_predictions1CI = get_prediction(x_val_seq,LAB =np.array(y_val_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(val_predictions1CI).count(1))
print("Class 0:", list(val_predictions1CI).count(0))

# ###################################################################
print("LAYER 2:")
test_suffixes2 = fingerprint_suffix_vals(x_test, layer2)
print("Suffixes computed for all test data")
print(np.shape(x_test),np.shape(y_test))
actual_preds2 = actual_preds1
test_predictions2 = test_predictions1
print("Predictions computed for all test data")

print("Class 1:", list(test_predictions2).count(1))
print("Class 0:", list(test_predictions2).count(0))

test_suffixes2F = fingerprint_suffix_vals(x_test_seq, layer2)
print("FEATURE Suffixes computed for all test data")
actual_preds2F = actual_preds1F
test_predictions2F = test_predictions1F
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(test_predictions2F).count(1))
print("Class 0 F:", list(test_predictions2F).count(0))

test_suffixes2FT = fingerprint_suffix_vals(x_test_seq, layer2)
print("FEATURE AND PROP Suffixes computed for all test data")
actual_preds2FT = actual_preds1FT
test_predictions2FT = test_predictions1FT
print("FEATURE AND PROP Predictions computed for all test data")

print("Class 0 ft:", list(test_predictions2FT).count(0))
print("Class 1 ft:", list(test_predictions2FT).count(1))
print("Class 2 ft:", list(test_predictions2FT).count(2))
print("Class 3 ft:", list(test_predictions2FT).count(3))

test_suffixes2CI = fingerprint_suffix_vals(x_test_seq, layer2)
print("PROP Suffixes computed for all test data")
actual_preds2CI, test_predictions2CI = get_prediction(x_test_seq,LAB =np.array(y_test_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(test_predictions2CI).count(1))
print("Class 0:", list(test_predictions2CI).count(0))

val_suffixes2F = fingerprint_suffix_vals(x_val_seq, layer2)
print("FEATURE Suffixes computed for all val data")
Vactual_preds2F, val_predictions2F = get_prediction(x_val_seq,LAB=np.array(y_val_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=Vthreshold1,THRESHOLD2=Vthreshold2)
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(val_predictions2F).count(1))
print("Class 0 F:", list(val_predictions2F).count(0))


val_suffixes2CI = fingerprint_suffix_vals(x_val_seq, layer2)
print("PROP Suffixes computed for all val data")
Vactual_preds2CI, val_predictions2CI = get_prediction(x_val_seq,LAB =np.array(y_val_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(val_predictions2CI).count(1))
print("Class 0:", list(val_predictions2CI).count(0))
# ###########################################################################
print("LAYER 3")
test_suffixes3 = fingerprint_suffix_vals(x_test, layer3)
print("Suffixes computed for all test data")
print(np.shape(x_test),np.shape(y_test))
actual_preds3 = actual_preds1
test_predictions3 = test_predictions1
print("Predictions computed for all test data")

print("Class 1:", list(test_predictions3).count(1))
print("Class 0:", list(test_predictions3).count(0))

test_suffixes3F = fingerprint_suffix_vals(x_test_seq, layer3)
print("FEATURE Suffixes computed for all test data")
actual_preds3F = actual_preds1F
test_predictions3F = test_predictions1F
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(test_predictions3F).count(1))
print("Class 0 F:", list(test_predictions3F).count(0))

test_suffixes3FT = fingerprint_suffix_vals(x_test_seq, layer3)
print("FEATURE AND PROP Suffixes computed for all test data")
actual_preds3FT = actual_preds1FT
test_predictions3FT = test_predictions1FT
print("FEATURE AND PROP Predictions computed for all test data")

print("Class 0 ft:", list(test_predictions3FT).count(0))
print("Class 1 ft:", list(test_predictions3FT).count(1))
print("Class 2 ft:", list(test_predictions3FT).count(2))
print("Class 3 ft:", list(test_predictions3FT).count(3))

test_suffixes3CI = fingerprint_suffix_vals(x_test_seq, layer3)
print("PROP Suffixes computed for all test data")
actual_preds3CI, test_predictions3CI = get_prediction(x_test_seq,LAB =np.array(y_test_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(test_predictions3CI).count(1))
print("Class 0:", list(test_predictions3CI).count(0))

val_suffixes3F = fingerprint_suffix_vals(x_val_seq, layer3)
print("FEATURE Suffixes computed for all val data")
Vactual_preds3F, val_predictions3F = get_prediction(x_val_seq,LAB=np.array(y_val_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=Vthreshold1,THRESHOLD2=Vthreshold2)
print("FEATURE Predictions computed for all val data")

print("Class 1 F:", list(val_predictions3F).count(1))
print("Class 0 F:", list(val_predictions3F).count(0))


val_suffixes3CI = fingerprint_suffix_vals(x_val_seq, layer3)
print("PRP Suffixes computed for all val data")
Vactual_preds3CI, val_predictions3CI = get_prediction(x_val_seq,LAB =np.array(y_val_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all val data")
print("Class 1:", list(val_predictions3CI).count(1))
print("Class 0:", list(val_predictions3CI).count(0))

##################################################################################
print("ALL")
test_suffixesA = fingerprint_suffix_vals(x_test, "ALL")
print("Suffixes computed for all test data")
print(np.shape(x_test),np.shape(y_test))
actual_predsA = actual_preds1
test_predictionsA = test_predictions1
print("Predictions computed for all test data")

test_suffixesAF = fingerprint_suffix_vals(x_test_seq, "ALL")
print("FEATURE Suffixes computed for all test data")
actual_predsAF, test_predictionsAF = get_prediction(x_test_seq,LAB=np.array(y_test_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=threshold1,THRESHOLD2=threshold2)
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(test_predictionsAF).count(1))
print("Class 0 F:", list(test_predictionsAF).count(0))


test_suffixesAFT = fingerprint_suffix_vals(x_test_seq, "ALL")
print("FEATURE AND PROP Suffixes computed for all test data")
actual_predsAFT, test_predictionsAFT = get_prediction(x_test_seq,LAB=np.array(y_test_seq),Feature=True,Prop=True,TEST=True,THRESHOLD1=threshold1,THRESHOLD2=threshold2)
print("FEATURE AND PROP Predictions computed for all test data")

print("Class 0 FT:", list(test_predictionsAFT).count(0))
print("Class 1 FT:", list(test_predictionsAFT).count(1))
print("Class 2 FT:", list(test_predictionsAFT).count(2))
print("Class 3 FT:", list(test_predictionsAFT).count(3))

test_suffixesACI = fingerprint_suffix_vals(x_test_seq, "ALL")
print("PROP Suffixes computed for all test data")
actual_predsACI, test_predictionsACI = get_prediction(x_test_seq,LAB =np.array(y_test_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(test_predictionsACI).count(1))
print("Class 0:", list(test_predictionsACI).count(0))

val_suffixesAF = fingerprint_suffix_vals(x_val_seq, "ALL")
print("FEATURE Suffixes computed for all val data")
Vactual_predsAF, val_predictionsAF = get_prediction(x_val_seq,LAB=np.array(y_val_seq),Feature=True,Prop=False,TEST=True,THRESHOLD1=Vthreshold1,THRESHOLD2=Vthreshold2)
print("FEATURE Predictions computed for all test data")

print("Class 1 F:", list(val_predictionsAF).count(1))
print("Class 0 F:", list(val_predictionsAF).count(0))


val_suffixesACI = fingerprint_suffix_vals(x_val_seq, "ALL")
print("PROP Suffixes computed for all val data")
Vactual_predsACI, val_predictionsACI = get_prediction(x_val_seq,LAB =np.array(y_val_seq),Feature=False,Prop=True,TEST=True)
print("PROP Predictions computed for all test data")
print("Class 1:", list(val_predictionsACI).count(1))
print("Class 0:", list(val_predictionsACI).count(0))



In [ ]:
# Generate Decision Trees 
basic_estimator1 = tree.DecisionTreeClassifier()
basic_estimator1.fit(test_suffixes1, test_predictions1)

basic_estimator1F = tree.DecisionTreeClassifier()
basic_estimator1F.fit(test_suffixes1F, test_predictions1F)

basic_estimator1FT = tree.DecisionTreeClassifier()
basic_estimator1FT.fit(test_suffixes1FT, test_predictions1FT)

basic_estimator1CI = tree.DecisionTreeClassifier()
basic_estimator1CI.fit(test_suffixes1CI, test_predictions1CI)
# # Basic decision tree
basic_estimator2 = tree.DecisionTreeClassifier()
basic_estimator2.fit(test_suffixes2, test_predictions2)

basic_estimator2F = tree.DecisionTreeClassifier()
basic_estimator2F.fit(test_suffixes2F, test_predictions2F)

basic_estimator2FT = tree.DecisionTreeClassifier()
basic_estimator2FT.fit(test_suffixes2FT, test_predictions2FT)


basic_estimator2CI = tree.DecisionTreeClassifier()
basic_estimator2CI.fit(test_suffixes2CI, test_predictions2CI)

basic_estimator3 = tree.DecisionTreeClassifier()
basic_estimator3.fit(test_suffixes3, test_predictions3)

basic_estimator3F = tree.DecisionTreeClassifier()
basic_estimator3F.fit(test_suffixes3F, test_predictions3F)

basic_estimator3FT = tree.DecisionTreeClassifier()
basic_estimator3FT.fit(test_suffixes3FT, test_predictions3FT)


basic_estimator3CI = tree.DecisionTreeClassifier()
basic_estimator3CI.fit(test_suffixes3CI, test_predictions3CI)

basic_estimatorA = tree.DecisionTreeClassifier()
basic_estimatorA.fit(test_suffixesA, test_predictionsA)

basic_estimatorAF = tree.DecisionTreeClassifier()
basic_estimatorAF.fit(test_suffixesAF, test_predictionsAF)

basic_estimatorAFT = tree.DecisionTreeClassifier()
basic_estimatorAFT.fit(test_suffixesAFT, test_predictionsAFT)


basic_estimatorACI = tree.DecisionTreeClassifier()
basic_estimatorACI.fit(test_suffixesACI, test_predictionsACI)


In [ ]:
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

# The `output` variable refer to the output of the model,
# so, in this case, `output` shape is `(1, 2)` i.e., (samples, classes).

#Loss and Model Modifier for middle layers
def loss_gen_sum(node_list):
    def loss(output):
        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_avg_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss

def loss_gen_separate(node_list):
    def loss(output):
        loss_val = tuple([output[ind][i] for ind,i in enumerate(node_list)])
        return loss_val
    return loss

def model_modifier_layer1(current_model):
    target_layer = current_model.get_layer(name="dense_1")
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer2(current_model):
    target_layer = current_model.get_layer(name="dense_2")
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer3(current_model):
    target_layer = current_model.get_layer(name="dense_3")
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

def loss_crosstrack(output):
    return (output[0][0])

def loss_heading(output):
    return (output[0][1])

def no_loss(output):
    return (output[0][0] * 0)


gradcam1 = GradcamPlusPlus(model,
                    model_modifier=model_modifier_layer1,
                    clone=True)
gradcam2 = GradcamPlusPlus(model,
                    model_modifier=model_modifier_layer2,
                    clone=True)
gradcam3 = GradcamPlusPlus(model,
                    model_modifier=model_modifier_layer3,
                    clone=True)

In [ ]:
def describe_invariants_all_labels_vals(all_invariants,prevlayer,layer,suffixes,label):

    min_support = 0
    
    subplot_args = { 'nrows': 1, 'ncols': 4, 'figsize': (12, 12),
                 'subplot_kw': {'xticks': [], 'yticks': []} }
    subplot_args2 = { 'nrows': 1, 'ncols': 3, 'figsize': (12, 12),
                 'subplot_kw': {'xticks': [], 'yticks': []} }

    print("\n PRINTING PURE RULES WITH SUPPORT MORE THAN {}:".format(min_support));
    #Pstr = ""
    for cl, invs in all_invariants.items():
       
        if ((cl == -1)): # (label == -1) and 
            continue
    
        for indx in range (0, len(invs)):
          
            inv = invs[indx]
           
            ## GET THE LIST OF INDICES OF INPUTS THAT SATISFY THE RULE
            cls = get_suffix_cluster_vals(inv[0],inv[1],suffixes)
      
            neurons = inv[0]
            signature = inv[1]

                
            Pstr = ""
            Pstr = Pstr + str("Class:" + str(cl) + ", Rule:(neurons:" + str(inv[0])  + ",signature:" + str(inv[1]) +"), Support:" + str(inv[2]));
            ## PRINT DETAILS REG THE RULE
            print(Pstr)
            
            ### FOR RULES FOR CLASS 1 (FEATURE PRESENT) , HIGHLIGHT PIXELS THAT IMPACT THE RULE USING GRADCAM.
            ### THIS IS A STEP TO VISUALLY VALIDATE IF THE RULE TRULY CORRESPONDS TO THE FEATURE
            if (cl != 1 or layer == "ALL"):
                continue
            heatmap_sequence = list()
            sum_cam = np.zeros((1,200,360),dtype=float)
            cnt_cam = 0
            for cls_indx in range(0,len(cls)):
                img_indx = cls[cls_indx]
                img = x_test_seq[img_indx]
                ops = y_test_seq[img_indx]
                img_mod = np.expand_dims(img/255.0,axis=0)
                
                if (layer == "dense_1"):
                    cam = gradcam1(loss_gen_sum(inv[0]),img_mod,penultimate_layer=-1)
                if (layer == "dense_2"):
                    cam = gradcam1(loss_gen_sum(inv[0]),img_mod,penultimate_layer=-1)
                if (layer == "dense_3"):
                    cam = gradcam1(loss_gen_sum(inv[0]),img_mod,penultimate_layer=-1)
                    
                cam = normalize(cam)
                sum_cam = sum_cam + cam
                cnt_cam = cnt_cam + 1
                ## PRINT THE SALIENCY MAP FOR EVERY 5TH IMAGE
                if (cls_indx % 5 == 0):
                    act_op = model.predict(np.expand_dims(img/255.0,axis=0))
                    
                    err_cte = np.squeeze((ops[0])-(act_op[0][0]*8.0))
                    err_he = np.squeeze((ops[1])-(act_op[0][1]*35.0))
                    #print("IDEAL - ACT OPS:", np.abs(np.round(err_cte,decimals=2)), np.abs(np.round(err_he,decimals=2)) )
                    
                    f1, ax1 = plt.subplots(**subplot_args)
                    ax1[0].set_title('IMAGE'+str(indx)+'_'+str(cls_indx))
                    ax1[0].imshow(img)
                 #   ax1[1].set_title('CAM '+str(np.abs(np.round(err_cte,decimals=2))) + '_' + str(np.abs(np.round(err_he,decimals=2))))
                    ax1[1].set_title('CAM ')
                    ax1[1].imshow(cam[0], cmap='jet', alpha=0.5)
                    
                
                    
                    
            avg_cam = sum_cam/float(cnt_cam)
            f2, ax2 = plt.subplots(**subplot_args)
            ax2[0].set_title('AVG CAM FOR RULE '+str(indx))
            ax2[0].imshow(avg_cam[0], cmap='jet', alpha=0.5)

    

In [ ]:
## PRINT DETAILS OF THE EXTRACTED RULES AFTER DENSE_1
###################################################


#invariants1 = get_all_invariants_vals(basic_estimator1)
#layer = 'dense_1'
#print("TEST DATA RULES FOR PROP ")
#describe_invariants_all_labels_vals(invariants1, None, layer, suffixes = test_suffixes1, label = test_predictions1)

invariants1F = get_all_invariants_vals(basic_estimator1F)
layer = 'dense_1'
print("TEST DATA RULES FOR FEATURE  ")
describe_invariants_all_labels_vals(invariants1F, None, layer, suffixes = test_suffixes1F, label = test_predictions1F)

#invariants1FT = get_all_invariants_vals(basic_estimator1FT)
#layer = 'dense_1'
#print("TEST DATA RULES FOR FEATURE AND PROP  ")
#describe_invariants_all_labels_vals(invariants1FT, None, layer, suffixes = test_suffixes1FT, label = test_predictions1FT)

#invariants1CI = get_all_invariants_vals(basic_estimator1CI)
#layer = 'dense_1'
#print("TEST DATA RULES FOR PROP ON SMALLER SET ")
#describe_invariants_all_labels_vals(invariants1CI, None, layer, suffixes = test_suffixes1CI, label = test_predictions1CI)

In [ ]:
## PRINT DETAILS OF THE EXTRACTED RULES AFTER DENSE_2
###################################################

#invariants2 = get_all_invariants_vals(basic_estimator2)
#layer = 'dense_2'
#print("TEST DATA RULES FOR PROP ")
#describe_invariants_all_labels_vals(invariants2, None, layer, suffixes = test_suffixes2, label = test_predictions2)

invariants2F = get_all_invariants_vals(basic_estimator2F)
layer = 'dense_2'
print("TEST DATA RULES FOR FEATURE  ")
describe_invariants_all_labels_vals(invariants2F, None, layer, suffixes = test_suffixes2F, label = test_predictions2F)

#invariants2FT = get_all_invariants_vals(basic_estimator2FT)
#layer = 'dense_2'
#print("TEST DATA RULES FOR FEATURE AND PROP  ")
#describe_invariants_all_labels_vals(invariants2FT, None, layer, suffixes = test_suffixes2FT, label = test_predictions2FT)

#invariants2CI = get_all_invariants_vals(basic_estimator2CI)
#layer = 'dense_2'
#print("TEST DATA RULES FOR PROP ON SMALLER SET ")
#describe_invariants_all_labels_vals(invariants2CI, None, layer, suffixes = test_suffixes2CI, label = test_predictions2CI)

In [ ]:
## PRINT DETAILS OF THE EXTRACTED RULES AFTER DENSE_3
###################################################

#invariants3 = get_all_invariants_vals(basic_estimator3)
#layer = 'dense_3'
#print("TEST DATA RULES FOR PROP ")
#describe_invariants_all_labels_vals(invariants3, None, layer, suffixes = test_suffixes3, label = test_predictions3)

invariants3F = get_all_invariants_vals(basic_estimator3F)
layer = 'dense_3'
print("TEST DATA RULES FOR FEATURE  ")
describe_invariants_all_labels_vals(invariants3F, None, layer, suffixes = test_suffixes3F, label = test_predictions3F)

#invariants3FT = get_all_invariants_vals(basic_estimator3FT)
#layer = 'dense_3'
#print("TEST DATA RULES FOR FEATURE AND PROP  ")
#describe_invariants_all_labels_vals(invariants3FT, None, layer, suffixes = test_suffixes3FT, label = test_predictions3FT)

#invariants3CI = get_all_invariants_vals(basic_estimator3CI)
#layer = 'dense_3'
#print("TEST DATA RULES FOR PROP ON SMALLER SET ")
#describe_invariants_all_labels_vals(invariants3CI, None, layer, suffixes = test_suffixes3CI, label = test_predictions3CI)

In [ ]:
## PRINT DETAILS OF THE EXTRACTED RULES USING ALL LAYERS
###################################################

#invariantsA = get_all_invariants_vals(basic_estimatorA)
#layer = 'ALL'
#print("TEST DATA RULES FOR PROP ")
#describe_invariants_all_labels_vals(invariantsA, None, layer, suffixes = test_suffixesA, label = test_predictionsA)

invariantsAF = get_all_invariants_vals(basic_estimatorAF)
layer = 'ALL'
print("TEST DATA RULES FOR FEATURE  ")
describe_invariants_all_labels_vals(invariantsAF, None, layer, suffixes = test_suffixesAF, label = test_predictionsAF)

#invariantsAFT = get_all_invariants_vals(basic_estimatorAFT)
#layer = 'ALL'
#print("TEST DATA RULES FOR FEATURE AND PROP  ")
#describe_invariants_all_labels_vals(invariantsAFT, None, layer, suffixes = test_suffixesAFT, label = test_predictionsAFT)

#invariantsACI = get_all_invariants_vals(basic_estimatorACI)
#layer = 'ALL'
#print("TEST DATA RULES FOR PROP ON SMALLER SET ")
#describe_invariants_all_labels_vals(invariantsACI, None, layer, suffixes = test_suffixesACI, label = test_predictionsACI)

In [ ]:
def check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   
   if (VAL == False):
     if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
     else:
        return False

   
   found = True
   oper = -1
   layer_vals = (layer_vals).flatten()
   
   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         #print(oper, v, vsig, val)
         if (oper == 0):
           if (val > vsig):
             found = False
             break
         else:
           if (val <= vsig):
            found = False
            break
         oper = -1
     if (found == True):
        break 

   if (found == False):
      return -1
   else:
     return ix 
    
def correlate(invariants1,clas1,invariants2,clas2,val_suffixes,val_preds_prop,val_preds_feat):
  
    neurons_list1 = []
    neurons_sig1 = []
    for cl, invs in invariants1.items():
        if ((cl == -1) or (clas1 != cl)): # (label == -1) and 
            continue
       
      
        for indx in range (0, len(invs)):
           
            inv = invs[indx]    
            neurons = []
            for indx in range(0,len(inv[0])):
                neurons.append(-1)
                neurons.append(inv[0][indx])

            neurons_list1.append(neurons)
            neurons_sig1.append(inv[1])
        
        
    neurons_list2= []
    neurons_sig2= []
    for cl, invs in invariants2.items():
        if ((cl == -1) or (clas2!= cl)): # (label == -1) and 
            continue
       
      
        for indx in range (0, len(invs)):
           
            inv = invs[indx]    
            neurons = []
            for indx in range(0,len(inv[0])):
                neurons.append(-1)
                neurons.append(inv[0][indx])

            neurons_list2.append(neurons)
            neurons_sig2.append(inv[1])
    
    print(len(neurons_list1),len(neurons_list2))
    feature_cnt = 0
    prop_cnt = 0
    for indx1 in range(0,len(val_suffixes)):
            prop = val_preds_prop[indx1]
            feat = val_preds_feat[indx1]
            match1 = check_pattern_inter(val_suffixes[indx1],val_suffixes[indx1],neurons_list1,neurons_sig1)
            match2 = check_pattern_inter(val_suffixes[indx1],val_suffixes[indx1],neurons_list2,neurons_sig2)

            if (match1 != -1):
                    if (feat == clas1):
                        feature_cnt = feature_cnt + 1
            
                        if (match2 != -1):
                            if (prop == clas2):
                                prop_cnt = prop_cnt + 1
         
    if (feature_cnt == 0):
        print("% of Feature CNT in Prop CNT:", 0)
    else:
        print("% of Feature CNT in Prop CNT:", float(prop_cnt/feature_cnt)*100)
                
         

def validate(invariants,clas,val_suffixes,val_preds_prop,val_preds_feat,Feature=-1,Prop=-1):
  
   
    for cl, invs in invariants.items():
        if ((cl == -1) or (clas != cl)): # (label == -1) and 
            continue
       
      
        for indx in range (0, len(invs)):
           
            inv = invs[indx]    
            neurons = []
            for indx in range(0,len(inv[0])):
                neurons.append(-1)
                neurons.append(inv[0][indx])

    
            recall = 0
            prec = 0
            for indx1 in range(0,len(val_suffixes)):
                match = check_pattern_inter(val_suffixes[indx1],val_suffixes[indx1],[neurons],[inv[1]])
                if (match != -1):
                    recall = recall + 1
                    prop = val_preds_prop[indx1]
                    feat = val_preds_feat[indx1]
                    if (Feature == 1):
                        if (feat != 1):
                            if (Prop == -1):
                                prec = prec + 1
                            else:
                                if (Prop == 1):
                                    if (prop == 1):
                                        prec= prec + 1
                                else:
                                    if (prop == 0):
                                        prec = prec + 1
                        continue
                                    
                    if (Feature == 0):
                        if (feat != 0):
                            if (Prop == -1):
                                prec = prec + 1
                            else:
                                if (Prop == 1):
                                    if (prop == 1):
                                        prec= prec+ 1
                                else:
                                    if (prop == 0):
                                        prec = prec+ 1

                        continue
                                    
                    if (Prop == 1):
                        if (prop != 0):
                            if (Feature == -1):
                                prec = prec + 1
                            else:
                                if (Feature == 1):
                                    if (feat == 0):
                                        prec = prec + 1
                                else:
                                    if (feat == 1):
                                        prec = prec + 1
                        continue
                    
                    if (Prop == 0):
                        if (prop != 1):
                            if (Feature == -1):
                                prec = prec + 1
                            else:
                                if (Feature == 1):
                                    if (feat == 0):
                                        prec = prec + 1
                                else:
                                    if (feat == 1):
                                        prec = prec + 1
                        continue
                                    
            
            if (recall == 0):
                print("CLASS;", cl, inv, ";Precision:",0,";Recall:",0)
            else:
                print("CLASS;", cl, inv, ";Precision:",float(prec/recall)*100,";Recall:",float(recall/len(val_suffixes))*100)
            

print("### LAYER 1")
print("Validate Invariants for Feature SAT")       
print("-----------")
validate(invariants1F,0,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=1,Prop=-1)      
print("-----------")
print("Validate Invariants for Feature UNSAT")     
print("-----------")
validate(invariants1F,1,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=0,Prop=-1) 

#print("-----------")
#print("Validate Invariants for Prop SAT")  
#print("-----------")
#validate(invariants1CI,1,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=-1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Prop UNSAT")     
#print("-----------")
#validate(invariants1CI,0,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=-1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop SAT")     
#print("-----------")
#validate(invariants1FT,0,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop SAT")      
#print("-----------")
#validate(invariants1FT,1,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=0,Prop=1) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop UNSAT")      
#print("-----------")
#validate(invariants1FT,2,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop UNSAT") 
#print("-----------")
#validate(invariants1FT,3,val_suffixes1F,val_predictions1CI,val_predictions1F,Feature=0,Prop=0) 
#print("-----------")

print("#### LAYER 2")
print("Validate Invariants for Feature SAT")       
print("-----------")
validate(invariants2F,0,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=1,Prop=-1)      
print("-----------")
print("Validate Invariants for Feature UNSAT")     
print("-----------")
validate(invariants2F,1,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=0,Prop=-1) 

#print("-----------")
#print("Validate Invariants for Prop SAT")  
#print("-----------")
#validate(invariants2CI,1,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=-1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Prop UNSAT")     
#print("-----------")
#validate(invariants2CI,0,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=-1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop SAT")     
#print("-----------")
#validate(invariants2FT,0,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop SAT")      
#print("-----------")
#validate(invariants2FT,1,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=0,Prop=1) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop UNSAT")      
#print("-----------")
#validate(invariants2FT,2,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop UNSAT") 
#print("-----------")
#validate(invariants2FT,3,val_suffixes2F,val_predictions2CI,val_predictions2F,Feature=0,Prop=0) 
#print("-----------")

print("### LAYER 3")
print("Validate Invariants for Feature SAT")       
print("-----------")
validate(invariants3F,0,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=1,Prop=-1)      
print("-----------")
print("Validate Invariants for Feature UNSAT")     
print("-----------")
validate(invariants3F,1,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=0,Prop=-1) 

#print("-----------")
#print("Validate Invariants for Prop SAT")  
#print("-----------")
#validate(invariants3CI,1,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=-1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Prop UNSAT")     
#print("-----------")
#validate(invariants3CI,0,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=-1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop SAT")     
#print("-----------")
#validate(invariants3FT,0,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop SAT")      
#print("-----------")
#validate(invariants3FT,1,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=0,Prop=1) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop UNSAT")      
#print("-----------")
#validate(invariants3FT,2,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop UNSAT") 
#print("-----------")
#validate(invariants3FT,3,val_suffixes3F,val_predictions3CI,val_predictions3F,Feature=0,Prop=0) 
#print("-----------")

print("### LAYER ALL")
print("Validate Invariants for Feature SAT")       
print("-----------")
validate(invariantsAF,0,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=1,Prop=-1)      
print("-----------")
print("Validate Invariants for Feature UNSAT")     
print("-----------")
validate(invariantsAF,1,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=0,Prop=-1) 

#print("-----------")
#print("Validate Invariants for Prop SAT")  
#print("-----------")
#validate(invariantsACI,1,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=-1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Prop UNSAT")     
#print("-----------")
#validate(invariantsACI,0,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=-1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop SAT")     
#print("-----------")
#validate(invariantsAFT,0,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=1,Prop=1)      
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop SAT")      
#print("-----------")
#validate(invariantsAFT,1,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=0,Prop=1) 
#print("-----------")
#print("Validate Invariants for Feature SAT and Prop UNSAT")      
#print("-----------")
#validate(invariantsAFT,2,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=1,Prop=0) 
#print("-----------")
#print("Validate Invariants for Feature UNSAT and Prop UNSAT") 
#print("-----------")
#validate(invariantsAFT,3,val_suffixesAF,val_predictionsACI,val_predictionsAF,Feature=0,Prop=0) 
#print("-----------")
        
        